In [4]:
'''
ManGo - Machine Gnostics Library
Copyright (C) 2025  ManGo Team

This work is licensed under the terms of the GNU General Public License version 3.0.

Description: Implementation of Gnostic Median calculations
'''

import numpy as np
from scipy.optimize import root_scalar

def calculate_gmedian(data, case='quantifying', z_range=None, tol=1e-8):
    """
    Calculate the Gnostic Median of a data sample.
    
    The G-median is defined as the value Z_med for which the sum of irrelevances equals zero.
    Implements both quantifying and estimating cases based on equations 14.23 and 14.24.
    
    Parameters
    ----------
    data : array-like
        Input data sample
    case : str, default='quantifying'
        The type of G-median to calculate:
        - 'quantifying': Uses equation 14.23
        - 'estimating': Uses equation 14.24
    z_range : tuple, optional
        Initial search range for Z_med (min, max). If None, will be determined from data
    tol : float, default=1e-8
        Tolerance for convergence
        
    Returns
    -------
    float
        The calculated G-median value
    """
    data = np.asarray(data)
    
    if z_range is None:
        z_range = (np.min(data), np.max(data))
    
    def irrelevance_sum(z_med):
        q = np.abs(data/z_med)**(1/2)  # equation 14.25
        
        if case == 'quantifying':
            # equation 14.23
            return np.sum((q**2 - 1/q**2) / 2)
        else:  # estimating case
            # equation 14.24
            return np.sum((q**2 - 1/q**2) / (q**2 + 1/q**2))
    
    # Find root of irrelevance sum to get G-median
    result = root_scalar(irrelevance_sum, 
                        bracket=z_range,
                        method='brentq',
                        rtol=tol)
    
    if not result.converged:
        raise RuntimeError("G-median calculation did not converge")
        
    return result.root

class GnosticMedian:
    """
    Class for calculating and analyzing Gnostic Medians.
    
    This class provides methods to compute G-medians and related metrics
    for both quantifying and estimating cases.
    """
    
    def __init__(self, data):
        """
        Initialize with a data sample.
        
        Parameters
        ----------
        data : array-like
            Input data sample
        """
        self.data = np.asarray(data)
        
    def compute(self, case='quantifying'):
        """
        Compute the G-median for the data sample.
        
        Parameters
        ----------
        case : str, default='quantifying'
            Type of G-median to calculate ('quantifying' or 'estimating')
            
        Returns
        -------
        float
            Calculated G-median value
        """
        return calculate_gmedian(self.data, case=case)
    
    def compute_both(self):
        """
        Compute both quantifying and estimating G-medians.
        
        Returns
        -------
        dict
            Dictionary containing both G-median values
        """
        return {
            'quantifying': self.compute(case='quantifying'),
            'estimating': self.compute(case='estimating')
        }

# Example usage:
if __name__ == "__main__":
    # Sample data
    data = np.array([1.2, 2.3, 3.4, 4.5, 5.6])
    
    # Using function directly
    g_median_quant = calculate_gmedian(data, case='e')
    print(f"Quantifying G-median: {g_median_quant:.4f}")
    
    # Using class
    gm = GnosticMedian(data)
    results = gm.compute_both()
    print("Both G-medians:", results)


Quantifying G-median: 3.0610
Both G-medians: {'quantifying': 2.9428037292669535, 'estimating': 3.0610164206764527}
